In [0]:
SELECT 
 DISTINCT PRODUCT_NAME
 ,PRODUCTGROUP_NAME
FROM pricing_analytics_vrs.silver.daily_pricing_silver
WHERE PRODUCT_NAME='Banana'

In [0]:
CREATE TABLE IF NOT EXISTS pricing_analytics_vrs.gold.reporting_dim_product_gold_SCDTYPE1
 LIKE pricing_analytics_vrs.gold.reporting_dim_product_gold


In [0]:
SELECT * FROM pricing_analytics_vrs.gold.reporting_dim_product_gold_SCDTYPE1
WHERE PRODUCT_NAME='Banana'

In [0]:
UPDATE pricing_analytics_vrs.silver.daily_pricing_silver
SET PRODUCTGROUP_NAME='Fruit',
lakehouse_updated_date = current_timestamp()
WHERE PRODUCT_NAME='Banana'

In [0]:
CREATE OR REPLACE TABLE pricing_analytics_vrs.silver.reporting_dim_product_stage_1 AS
SELECT 
 DISTINCT PRODUCT_NAME
 ,PRODUCTGROUP_NAME
FROM pricing_analytics_vrs.silver.daily_pricing_silver
WHERE lakehouse_updated_date > (SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM pricing_analytics_vrs.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE process_name = 'reportingDimensionTablesLoadScdType1' AND process_status = 'Completed' );

In [0]:
SELECT * FROM pricing_analytics_vrs.silver.reporting_dim_product_stage_1 

In [0]:
CREATE OR REPLACE TABLE pricing_analytics_vrs.silver.reporting_dim_product_stage_2 AS 
SELECT 
  silverDim.PRODUCT_NAME
  ,silverDim.PRODUCTGROUP_NAME
  ,goldDim.PRODUCT_NAME AS GOLD_PRODUCT_NAME
 ,CASE WHEN goldDim.PRODUCT_NAME IS NULL
 THEN ROW_NUMBER() OVER (  ORDER BY silverDim.PRODUCT_NAME,silverDim.PRODUCTGROUP_NAME) 
 ELSE goldDim.PRODUCT_ID END as PRODUCT_ID
 ,current_timestamp() as lakehouse_inserted_date
 ,current_timestamp() as lakehouse_updated_date
FROM pricing_analytics_vrs.silver.reporting_dim_product_stage_1 silverDim
LEFT OUTER JOIN pricing_analytics_vrs.gold.reporting_dim_product_gold_SCDTYPE1 goldDim
ON silverDim.PRODUCT_NAME= goldDim.PRODUCT_NAME
WHERE goldDim.PRODUCT_NAME IS NULL OR silverDim.PRODUCTGROUP_NAME <> goldDim.PRODUCTGROUP_NAME

In [0]:
SELECT * FROM pricing_analytics_vrs.silver.reporting_dim_product_stage_2

In [0]:
CREATE OR REPLACE TABLE pricing_analytics_vrs.silver.reporting_dim_product_stage_3 AS 
SELECT
  silverDim.PRODUCTGROUP_NAME
  ,silverDim.PRODUCT_NAME
,CASE WHEN GOLD_PRODUCT_NAME IS NULL
THEN silverDim.PRODUCT_ID + PREV_MAX_SK_ID 
ELSE PRODUCT_ID END as PRODUCT_ID
,current_timestamp() as lakehouse_inserted_date
,current_timestamp() as lakehouse_updated_date
FROM 
pricing_analytics_vrs.silver.reporting_dim_product_stage_2 silverDim
CROSS JOIN (SELECT nvl(MAX(PRODUCT_ID),0) as PREV_MAX_SK_ID FROM pricing_analytics_vrs.gold.reporting_dim_product_gold_SCDTYPE1 ) goldDim;

In [0]:
SELECT * FROM pricing_analytics_vrs.silver.reporting_dim_product_stage_3

In [0]:
MERGE INTO pricing_analytics_vrs.gold.reporting_dim_product_gold_SCDTYPE1 goldDim
USING pricing_analytics_vrs.silver.reporting_dim_product_stage_3 silverDim
ON goldDim.PRODUCT_NAME = silverDim.PRODUCT_NAME
WHEN MATCHED THEN 
UPDATE SET goldDim.PRODUCTGROUP_NAME=silverDim.PRODUCTGROUP_NAME
           ,goldDim.lakehouse_updated_date=current_timestamp()
WHEN NOT MATCHED THEN
INSERT *

In [0]:
SELECT * FROM pricing_analytics_vrs.gold.reporting_dim_product_gold_SCDTYPE1
WHERE PRODUCT_NAME='Banana'

In [0]:
INSERT INTO  pricing_analytics_vrs.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME,PROCESSED_TABLE_DATETIME,PROCESS_STATUS)
SELECT 'reportingDimensionTablesLoadScdType1' , max(lakehouse_updated_date) ,'Completed' FROM pricing_analytics_vrs.silver.daily_pricing_silver